# Differential gene expression
### Prepare environment
----

In [3]:
 
library('tximport', quietly=T)
library('DESeq2',quietly=T)
library('ashr',quietly=T)
library('tibble',quietly=T)
library('tidyverse',quietly=T)
library('Glimma',quietly=T)
library('RCurl',quietly=T)


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min



Attaching package: ‘S4Vectors’


The following object is masked from ‘package:utils’:

    findMatches


The following objects are masked from ‘package:base’:

    expand.grid, I, unname



Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
 

ERROR: Error in library("Glimma", quietly = T): there is no package called ‘Glimma’


# This notebook will create and save two data types:
1. matrix of significant differentially expressed orf's from the results of the differential expression analysis
2. matrix of VSD normalized counts ordered by variance across samples
## Both of these files will be created for transcript-level data and "gene-level" data using only transcripts with Kegg annotations and summing by Kegg annotation. 
#### The second method will result in data with rownames as Kegg annotations, meaning each annotation appears once in the matrix. 

# 1. Read in Salmon Counts
----

In [5]:
#fixing tximport function 

pattern='[[:alpha:]]+([[:digit:]]{2}|_)(_9[[:alpha:]]|[[:alpha:]]*)'

read.in <- function(org){
    dir <- paste("/work/nclab/lucy/SAB/Assembly/",org,"/salmon",sep='')
    files <- file.path(dir,list.files(dir,pattern=".sf",recursive=TRUE))
    
    names(files)=str_extract(files,pattern)
    names(files)=str_replace(names(files),'oFe', 'pFe') #correct a sample for 08 from oFe to pFe

    if (all(file.exists(files)) == FALSE) {
        print("ERROR IN FILE NAMES, not all files exist")
        print(paste("Directory:", dir, sep="/n"))
        print(paste("Files:", files, sep='/n'))
    }
    
    raw_counts <- tximport(files, type='salmon', txOut = TRUE) 
}

create.metadata=function(org){
    dir <- paste("/work/nclab/lucy/SAB/Assembly/",org,"/salmon",sep='')
    files <- file.path(dir,list.files(dir,pattern=".sf",recursive=TRUE))
    
    id=str_extract(files,pattern)
    id=str_replace(id,'oFe', 'pFe')
    metadata=data.frame('id'=id,'isolate'=org,'treatment'=str_extract(id,'[[:alpha:]]+(19|21_9|_back)'),'rep'=str_extract(id, 'A|B|C'))
    metadata$treatment=str_replace_all(metadata$treatment, c('pFe19'='High_Iron', 'pFe21_9'='Low_Iron','add_back'='Add_Back'))
    metadata
    print(metadata)                    
}
counts_4=read.in('04')
metadata_4=create.metadata('04')

counts_8=read.in('08')
metadata_8=create.metadata('08')

counts_6=read.in('06')
metadata_6=create.metadata('06')

counts_13=read.in('13')
metadata_13=create.metadata('13')
head(counts_8$counts)

reading in files with read_tsv

1 
2 
3 
4 
5 




         id isolate treatment rep
1 add_backA      04  Add_Back   A
2 add_backB      04  Add_Back   B
3  pFe21_9A      04  Low_Iron   A
4  pFe21_9B      04  Low_Iron   B
5  pFe21_9C      04  Low_Iron   C


reading in files with read_tsv

1 
2 
3 
4 
5 
6 
7 
8 




         id isolate treatment rep
1 add_backB      08  Add_Back   B
2 add_backC      08  Add_Back   C
3  pFe21_9A      08  Low_Iron   A
4    pFe19A      08 High_Iron   A
5    pFe19B      08 High_Iron   B
6    pFe19C      08 High_Iron   C
7  pFe21_9B      08  Low_Iron   B
8  pFe21_9C      08  Low_Iron   C


reading in files with read_tsv

1 
2 
3 
4 
5 
6 
7 
8 
9 




         id isolate treatment rep
1 add_backA      06  Add_Back   A
2 add_backB      06  Add_Back   B
3 add_backC      06  Add_Back   C
4    pFe19A      06 High_Iron   A
5    pFe19B      06 High_Iron   B
6    pFe19C      06 High_Iron   C
7  pFe21_9A      06  Low_Iron   A
8  pFe21_9B      06  Low_Iron   B
9  pFe21_9C      06  Low_Iron   C


reading in files with read_tsv

1 
2 
3 
4 
5 
6 
7 
8 
9 




         id isolate treatment rep
1 add_backA      13  Add_Back   A
2 add_backB      13  Add_Back   B
3 add_backC      13  Add_Back   C
4    pFe19A      13 High_Iron   A
5    pFe19B      13 High_Iron   B
6    pFe19C      13 High_Iron   C
7  pFe21_9A      13  Low_Iron   A
8  pFe21_9B      13  Low_Iron   B
9  pFe21_9C      13  Low_Iron   C


,add_backB,add_backC,pFe21_9A,pFe19A,pFe19B,pFe19C,pFe21_9B,pFe21_9C
NODE_10000_length_1734_cov_7.947622_g6638_i0.p1,34.000,15.000,9.000,5.000,7.000,15.000,10.000,1.000
NODE_10001_length_1733_cov_212.025904_g6639_i0.p1,677.000,263.878,49.000,37.000,30.000,58.000,108.000,141.237
NODE_10002_length_1733_cov_138.864458_g1003_i6.p1,165.923,88.899,28.753,14.298,16.228,44.130,12.042,26.511
NODE_10003_length_1733_cov_40.357229_g5975_i1.p2,6.055,1.363,1.001,1.098,0.000,3.387,9.037,4.809
NODE_10003_length_1733_cov_40.357229_g5975_i1.p1,4.000,2.000,4.000,1.000,0.000,7.000,4.000,1.000
NODE_10004_length_1733_cov_25.593373_g6640_i0.p1,70.000,40.000,11.000,11.000,5.000,9.000,34.000,17.000


### wrong and old read in function
read.in <- function(org,meta){
    dir <- paste("/work/nclab/lucy/SAB/Assembly/",org,"/salmon",sep='')
    
    counts_metadata <- meta
    files <- file.path(dir,list.files(dir,pattern=".sf",recursive=TRUE))
    
    names(files) <- paste0(counts_metadata$id)

    if (all(file.exists(files)) == FALSE) {
        print("ERROR IN FILE NAMES, not all files exist")
        print(paste("Directory:", dir, sep="/n"))
        print(paste("Files:", files, sep='/n'))
    }
    
    raw_counts <- tximport(files, type='salmon', txOut = TRUE) #should this be true? transcript level data?
    if (all(rownames(counts_metadata)==colnames(raw_counts)) == FALSE){
        print("ERROR MISMATCH IN METADATA AND COUNTS")
        print(paste("Metadata rows:", rownames(counts_metadata, sep='/n')))
        print(paste("Counts columns:", colnames(raw_counts), sep="\n"))
    }
    
    rownames(counts_metadata) <- colnames(raw_counts$counts)
    print(colnames(raw_counts$counts))
    raw_counts
}

counts_metadata4 <- data.frame(
    row.names = c("4add_backA",
         "4add_backB",
         "4pFe21_9A",
         "4pFe21_9B",
         "4pFe21_9C"),
    id=c("4add_backA",
         "4add_backB",
         "4pFe21_9A",
         "4pFe21_9B",
         "4pFe21_9C"),
    isolate=rep_len(4,5),
    treatment=c(rep_len("Iron_ammendment",2),rep_len("Low_Iron",3)),
    replicate=c('A','B','A','B','C'))

counts_metadata8 <- data.frame(
    row.names = c("8add_backB",
         "8add_backC",
         "8pFe21_9A",
         "8pFe19_A",
         "8pFe19_B",
         "8pFe19_C",
         "8pFe21_9B",
         "8pFe21_9C"),
    id=c("8add_backB",
         "8add_backC",
         "8pFe21_9A",
         "8pFe19_A",
         "8pFe19_B",
         "8pFe19_C",
         "8pFe21_9B",
         "8pFe21_9C"),
    isolate=rep_len(8,8),
    treatment=c(rep_len("Iron_ammendment",2), "Low_Iron", rep_len("High_Iron",3),rep_len("Low_Iron",2)),
    replicate=c('B','C','A','A','B','C','B','C'))

counts_metadata6 <- data.frame(
    row.names = c(
        "6add_backA",
         "6add_backB",
         "6add_backC",
         "6pFe21_9A",
         "6pFe21_9B",
         "6pFe21_9C",
         "6pFe19_A", 
         "6pFe19_B", 
         "6pFe19_C"),
    id=c(
        "6add_backA",
         "6add_backB",
         "6add_backC",
         "6pFe21_9A",
         "6pFe21_9B",
         "6pFe21_9C",
         "6pFe19_A", 
         "6pFe19_B", 
         "6pFe19_C"),
    isolate=rep_len(6,9),
    treatment=c(rep_len("Iron_ammendment",3),rep_len("Low_Iron",3), rep_len("High_Iron",3)),
    replicate=c(rep(c('A','B','C'),3)))

counts_metadata13 <- data.frame(
    row.names = c("13add_backA",
         "13add_backB",
         "13add_backC",
         "13pFe21_9A",
         "13pFe21_9B",
         "13pFe21_9C",
        "13pFe19_A", 
         "13pFe19_B",
         "13pFe19_C"),
    id=c("13add_backA",
         "13add_backB",
         "13add_backC",
         "13pFe21_9A",
         "13pFe21_9B",
         "13pFe21_9C",
        "13pFe19_A", 
         "13pFe19_B",
         "13pFe19_C"),
    isolate=rep_len(13,9),
    treatment=c(rep_len("Iron_ammendment",3),rep_len("Low_Iron",3), rep_len("High_Iron",3)),
    replicate=c(rep(c('A','B','C'),3)))


# 2. Create DeSeq object
---------
## Create deseq2 object normalize the counts object for plotting later, and run the differential expression
We should have four kinds of objects for each sample now, dds (deseq object), normalized dds, differential expression analysis object (de), and the differential expression results (res)
## 2.1 create dds

In [3]:
dds <- function(raw_counts, metadata, n){
    dds <- DESeqDataSetFromTximport( raw_counts,
                             colData=metadata,
                             design=~treatment)
    dds$treatment <- relevel(dds$treatment, ref = "Low_Iron")
    keep <- rowSums(counts(dds) >=5) >= n #filter out rows with too low expression
    print(nrow(dds))
    dds <- dds[keep, ]
    print(nrow(dds))
    dds
}

dds4 <- dds(counts_4, metadata_4, 2)
dds8 <- dds(counts_8, metadata_8, 2)
dds6 <- dds(counts_6, metadata_6, 3)
dds13 <- dds(counts_13, metadata_13, 3)
head(assays(dds13))

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
using counts and average transcript lengths from tximport



[1] 40907
[1] 27159


Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
using counts and average transcript lengths from tximport



[1] 37958
[1] 23714


Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
using counts and average transcript lengths from tximport



[1] 53342
[1] 35097


Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
using counts and average transcript lengths from tximport



[1] 48089
[1] 29014


List of length 2
names(2): counts avgTxLength

In [4]:
vsd.norm <- function(dds){
    vst(dds,blind=FALSE)
    }

#full vsd deseq2 objects:
vsd4 <- vsd.norm(dds4)
vsd8 <- vsd.norm(dds8)
vsd6 <- vsd.norm(dds6)
vsd13 <- vsd.norm(dds13)
colData(vsd13)

using 'avgTxLength' from assays(dds), correcting for library size

using 'avgTxLength' from assays(dds), correcting for library size

using 'avgTxLength' from assays(dds), correcting for library size

using 'avgTxLength' from assays(dds), correcting for library size



DataFrame with 9 rows and 4 columns
                   id     isolate treatment         rep
          <character> <character>  <factor> <character>
add_backA   add_backA          13 Add_Back            A
add_backB   add_backB          13 Add_Back            B
add_backC   add_backC          13 Add_Back            C
pFe19A         pFe19A          13 High_Iron           A
pFe19B         pFe19B          13 High_Iron           B
pFe19C         pFe19C          13 High_Iron           C
pFe21_9A     pFe21_9A          13 Low_Iron            A
pFe21_9B     pFe21_9B          13 Low_Iron            B
pFe21_9C     pFe21_9C          13 Low_Iron            C

## Below are flavodoxin genes to look for with kegg annotations to make sure they are not lost later

In [5]:

#isip2a:

d13=as.data.frame(assay(dds13)) %>% rownames_to_column('orfs')

filter(d13, orfs=='NODE_19011_length_1508_cov_60.425087_g9146_i1.p1') 
filter(d13, orfs=='NODE_32738_length_942_cov_54.683544_g18313_i0.p1')
filter(d13, orfs=='NODE_34623_length_873_cov_1933.596250_g19622_i0.p1')

orfs,add_backA,add_backB,add_backC,pFe19A,pFe19B,pFe19C,pFe21_9A,pFe21_9B,pFe21_9C
<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
NODE_19011_length_1508_cov_60.425087_g9146_i1.p1,42,60,3,116,103,64,38,35,22


orfs,add_backA,add_backB,add_backC,pFe19A,pFe19B,pFe19C,pFe21_9A,pFe21_9B,pFe21_9C
<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
NODE_32738_length_942_cov_54.683544_g18313_i0.p1,0,0,0,0,0,0,102,121,97


orfs,add_backA,add_backB,add_backC,pFe19A,pFe19B,pFe19C,pFe21_9A,pFe21_9B,pFe21_9C
<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
NODE_34623_length_873_cov_1933.596250_g19622_i0.p1,0,0,0,0,0,0,3619,3036,2551


In [5]:
ko4_def <- read.csv("ko4_def.csv")
ko8_def <- read.csv("ko8_def.csv")
ko6_def <- read.csv("ko6_def.csv")
ko13_def <- read.csv("ko13_def.csv")

# possible place for ko error where important genes dissapear

In [55]:

#add kegg annotation as metadata to rows
add.rowData <- function(dds, ko_def){
    salmon.orfs <- rownames(dds)
    head(salmon.orfs)
    ko <- filter(ko_def, (ko_def$orfs %in% salmon.orfs)==TRUE)

        #remove enzymes when shown
    ko$name <- gsub(" \\[EC.*\\]", "", ko$name)
    print(head(ko))
        # take ko's defined by name and symbol and collaps by orf's so each orf is shown once
    ko.sum <- ko %>% 
                     group_by(orfs)%>% 
                     summarize('ko_id'=str_flatten(ko_id, collapse='; '),
                              'symbol'=str_flatten(symbol, collaps="; "), 
                                'name'= str_flatten(name, collaps="; "))

        # create df of orf's not included in kegg annotation table
    ko.na <- data.frame('orfs'=salmon.orfs[(salmon.orfs %in% ko.sum$orfs)==FALSE], 'ko_id'=NA, 'symbol'=NA, 'name'=NA)
    full.anno <- rbind(ko.sum, ko.na ) %>% select(!orfs)
    mcols(dds) <- cbind(mcols(dds), full.anno)
    print(head(mcols(dds)))
}


mcols(dds4) <- add.rowData(dds4, ko4_def)
mcols(dds8) <- add.rowData(dds8, ko8_def)
mcols(dds6) <- add.rowData(dds6, ko6_def)
mcols(dds13) <- add.rowData(dds13, ko13_def)

                                              orfs ko_iteration  ko_id
1 NODE_10051_length_2055_cov_12.485368_g5395_i0.p1        ko:_1 K02014
2 NODE_10060_length_2054_cov_21.521454_g5168_i2.p1        ko:_1 K14683
3 NODE_10077_length_2052_cov_55.663466_g5412_i0.p1        ko:_1 K03320
4 NODE_10077_length_2052_cov_55.663466_g5412_i0.p1        ko:_2 K07573
5 NODE_10189_length_2039_cov_48.328077_g5484_i0.p1        ko:_1 K02575
6 NODE_10218_length_2035_cov_11.646279_g5502_i0.p2        ko:_1 K21867
                   symbol
1               TC.FEV.OM
2       SLC34A, NPT, nptA
3           amt, AMT, MEP
4            CSL4, EXOSC1
5  NRT2, narK, nrtP, nasA
6    AKT, KAT, GORK, SKOR
                                                                  name
1                          iron complex outermembrane recepter protein
2  solute carrier family 34 (sodium-dependent phosphate cotransporter)
3                                     ammonium transporter, Amt family
4                                    

ERROR: Error in .local(x, ..., value = value): 6 rows in value to replace 27159rows


# running process for orfs with annotations only

In [7]:

#add kegg annotation as metadata to rows
select.ko <- function(dds, ko_def){
    salmon.orfs <- rownames(dds)
    ko <- filter(ko_def, (ko_def$orfs %in% salmon.orfs)==TRUE)

        #remove enzymes when shown
    ko$name <- gsub(" \\[EC.*\\]", "", ko$name)
    
        # take ko's defined by name and symbol and collaps by orf's so each orf is shown once
    ko.sum <- ko %>% 
                     group_by(orfs)%>% 
                     summarize('ko_id'=str_flatten(ko_id, collapse='; '),
                              'symbol'=str_flatten(symbol, collaps="; "), 
                                'name'= str_flatten(name, collaps="; "))
    #dds.ko <- dds[(salmon.orfs %in% ko.sum$orfs)==TRUE, ]
    #ko.data <- ko.sum %>% select(!orfs)
    #mcols(dds.ko) <- cbind(mcols(dds.ko), ko.data)
    #dds.ko
}

ko.sum4 <- select.ko(dds4, ko4_def)
dds4.ko <- select.ko(dds4, ko4_def)
dds8.ko <- select.ko(dds8, ko8_def)
dds6.ko <- select.ko(dds6, ko6_def)
dds13.ko <- select.ko(dds13, ko13_def)

### 2.2 VSD Normalize counts 
---
Order rows by variance across treatment. Top rows will have highest variance in normalized counts between treatments. Save in vsd folder. 

In [8]:
vsd.norm <- function(dds){
    vst(dds,blind=FALSE)
    }

#full vsd deseq2 objects:
vsd4 <- vsd.norm(dds4)
vsd8 <- vsd.norm(dds8)
vsd6 <- vsd.norm(dds6)
vsd13 <- vsd.norm(dds13)

## order the df's by decreasing variance. top rows have highest varience between 
## samples. write dataframe
write.vsd <- function(vsd, org){
    vsd <- assay(vsd)
    vsd_order <- order(rowVars(vsd), decreasing=T)
    vsd_new <- vsd[vsd_order, ]
    print(paste('saving',org,sep=' '))
    vsd_new <- as.data.frame(vsd_new) %>% rownames_to_column("orfs")
    write.csv(vsd_new, paste('./vsd_files/', org, "vsd.csv", sep=""), row.names=FALSE)
}

write.vsd(vsd4, "04")
write.vsd(vsd8, "08")
write.vsd(vsd6, "06")
write.vsd(vsd13, "13")

using 'avgTxLength' from assays(dds), correcting for library size

using 'avgTxLength' from assays(dds), correcting for library size

using 'avgTxLength' from assays(dds), correcting for library size

using 'avgTxLength' from assays(dds), correcting for library size



[1] "saving 04"
[1] "saving 08"
[1] "saving 06"
[1] "saving 13"


In [57]:
#distance matrix 
vsd.dist.mat <- function(vsd){
    vsd_dist <- dist(t(assay(vsd)))
    vsd_dist_mat <- as.matrix(vsd_dist)
    rownames(vsd_dist_mat) <- paste(vsd$id)
    pheatmap(vsd_dist_mat,
            clustering_distance_rows = vsd_dist,
            clustering_distance_cols = vsd_dist,
            col = colors)
    }

dist_mat4 <- vsd.dist.mat(vsd4)
dist_mat8 <- vsd.dist.mat(vsd8)
dist_mat6 <- vsd.dist.mat(vsd6)
dist_mat13 <- vsd.dist.mat(vsd13)

grid.arrange(grobs=list(
    dist_mat4[[4]],
    dist_mat8[[4]], 
    dist_mat6[[4]], 
    dist_mat13[[4]]),
    ncol=2, nrow=2, common.legend=TRUE)

ERROR: Error in pheatmap(vsd_dist_mat, clustering_distance_rows = vsd_dist, clustering_distance_cols = vsd_dist, : could not find function "pheatmap"


## 2.3 Differential Expression from DESeq
---
Run the differential expression. 

Extract the results using res() and specifying the contrast, or which treatments we wish to compare.  

In [9]:
de4 <- DESeq(dds4)
de8 <- DESeq(dds8)
de6 <- DESeq(dds6)
de13 <- DESeq(dds13)

estimating size factors

using 'avgTxLength' from assays(dds), correcting for library size

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

estimating size factors

using 'avgTxLength' from assays(dds), correcting for library size

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

estimating size factors

using 'avgTxLength' from assays(dds), correcting for library size

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

estimating size factors

using 'avgTxLength' from assays(dds), correcting for library size

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



In [10]:

de4 <- DESeq(dds4)
de8 <- DESeq(dds8)
de6 <- DESeq(dds6)
de13 <- DESeq(dds13)

#contrasts
HvL <- c("treatment", "High_Iron", "Low_Iron")
AvL <- c("treatment", "Add_Back", "Low_Iron")
AvH <- c("treatment", "Add_Back", "High_Iron")

#results from Low Iron vs Iron ammendment
AvL4 <- results(de4, contrast=AvL, tidy=TRUE)
AvL8 <- results(de8, contrast=AvL, tidy=T)
AvL6 <- results(de6, contrast=AvL, tidy=TRUE)
AvL13 <- results(de13, contrast=AvL, tidy=TRUE)

#results from High Iron vs Low Iron
HvL8  <- results(de8, contrast=HvL, tidy=TRUE)
HvL6  <- results(de6, contrast=HvL,  tidy=TRUE)
HvL13  <- results(de13,contrast=HvL, tidy=TRUE)

#results from High Iron vs Iron ammendment
AvH8  <- results(de8, contrast=AvH, tidy=TRUE)
AvH6  <- results(de6, contrast=AvH,  tidy=TRUE)
AvH13  <- results(de13, contrast=AvH, tidy=TRUE)


#sum(res$padj < 0.05, na.rm=T) #sum up how many adjusted p-values are significant
#nrow(res)
#nrow(emap) ##??????how is the rows of emap > res?? Is this because res only has diff expressed contigs? 

colnames(AvL4)[1]  <- "orfs"
colnames(AvL8)[1]  <- "orfs"
colnames(AvL6)[1]  <- "orfs"
colnames(AvL13)[1]  <- "orfs"

colnames(HvL8)[1]  <- "orfs"
colnames(HvL6)[1]  <- "orfs"
colnames(HvL13)[1]  <- "orfs"

colnames(AvH8)[1]  <- "orfs"
colnames(AvH6)[1]  <- "orfs"
colnames(AvH13)[1]  <- "orfs"

estimating size factors

using 'avgTxLength' from assays(dds), correcting for library size

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

estimating size factors

using 'avgTxLength' from assays(dds), correcting for library size

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

estimating size factors

using 'avgTxLength' from assays(dds), correcting for library size

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

estimating size factors

using 'avgTxLength' from assays(dds), correcting for library size

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



In [13]:
head(AvL4)

,orfs,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,NODE_10000_length_2060_cov_11.205335_g5364_i0.p1,17.815316,0.4643168,0.9386188,0.4946809,0.620825381,0.90208279
2,NODE_10001_length_2060_cov_5.581278_g5365_i0.p1,15.834006,0.1955311,1.1688008,0.1672921,0.867140223,0.97272842
3,NODE_10002_length_2059_cov_37.920947_g5366_i0.p1,85.381955,-1.6125740,0.4985678,-3.2344127,0.001218932,0.03408861
4,NODE_10002_length_2059_cov_37.920947_g5366_i0.p2,7.518903,-0.3956059,1.2030317,-0.3288408,0.742276024,NA
5,NODE_10003_length_2059_cov_28.617321_g5366_i1.p1,25.249582,-1.2007758,0.8319401,-1.4433440,0.148923602,0.55676320
6,NODE_10004_length_2059_cov_25.532729_g4469_i1.p1,21.393023,1.4248841,0.6729055,2.1175101,0.034216582,0.27807968


In [17]:
# how many DE's per treatment-sample?
avl4.perc=(nrow(filter(AvL4,(padj < 0.05)==T))/nrow(de4))*100
avl4=nrow(filter(AvL4,(padj < 0.05)==T))

avl8.perc=(nrow(filter(AvL8,(padj < 0.05)==T))/nrow(de8))*100
avl8=nrow(filter(AvL8,(padj < 0.05)==T))

avl6.perc=(nrow(filter(AvL6,(padj < 0.05)==T))/nrow(de6))*100
avl6=nrow(filter(AvL6,(padj < 0.05)==T))

avl13.perc=(nrow(filter(AvL13,(padj < 0.05)==T))/nrow(de13))*100
avl13=nrow(filter(AvL13,(padj < 0.05)==T))

hvl8.perc=(nrow(filter(HvL8,(padj < 0.05)==T))/nrow(de8))*100
hvl8=nrow(filter(HvL8,(padj < 0.05)==T))

hvl6.perc=(nrow(filter(HvL6,(padj < 0.05)==T))/nrow(de6))*100
hvl6=nrow(filter(HvL6,(padj < 0.05)==T))

hvl13.perc=(nrow(filter(HvL13,(padj < 0.05)==T))/nrow(de13))*100
hvl13=nrow(filter(HvL13,(padj < 0.05)==T))

avh8.perc=(nrow(filter(AvH8,(padj < 0.05)==T))/nrow(de8))*100
avh8=nrow(filter(AvH8,(padj < 0.05)==T))

avh6.perc=(nrow(filter(AvH6,(padj < 0.05)==T))/nrow(de6))*100
avh6=nrow(filter(AvH6,(padj < 0.05)==T))

avh13.perc=(nrow(filter(AvH13,(padj < 0.05)==T))/nrow(de13))*100
avh13=nrow(filter(AvH13,(padj < 0.05)==T))

de_output = data.frame('Organism'=c('C. closterium 4', 'C. closterium 8', 'G. oceanica', 'G. huxleyi'),
                       'Iron amendment vs low iron'=c(avl4,avl8,avl6,avl13),
                       'High iron vs low iron'=c('NA', hvl8,hvl6,hvl13),
                       'Iron amendment vs high iron'=c('NA',avh8,avh6,avh13))

de_output_perc = data.frame('Organism'=c('C. closterium 4', 'C. closterium 8', 'G. oceanica', 'G. huxleyi'),
                       'Iron amendment vs low iron'=c(avl4.perc,avl8.perc,avl6.perc,avl13.perc),
                       'High iron vs low iron'=c('NA', hvl8.perc,hvl6.perc,hvl13.perc),
                       'Iron amendment vs high iron'=c('NA',avh8.perc,avh6.perc,avh13.perc)) 

write.csv(de_output, './de_res_files/de_output.csv', row.names=F)
write.csv(de_output_perc, './de_res_files/de_output_perc.csv', row.names=F)



## Logfold 2 shrinkage

In [12]:

lfc4.AvL <- lfcShrink(de4,  contrast=c("treatment", "Add_Back", "Low_Iron"), type='ashr')
lfc4.AvL <- lfc4.AvL %>% as.data.frame() %>% rownames_to_column("orfs") 
write.csv(lfc4.AvL, "./de_res_files/lfc4.AvL.csv", row.names=F)

lfc8.AvL <- lfcShrink(de8, contrast=c("treatment", "Add_Back", "Low_Iron"), type='ashr')
lfc8.AvL <- lfc8.AvL %>% as.data.frame() %>% rownames_to_column("orfs") 
write.csv(lfc8.AvL, "./de_res_files/lfc8.AvL.csv", row.names=F)

lfc6.AvL <- lfcShrink(de6, contrast=c("treatment", "Add_Back", "Low_Iron"), type='ashr')
lfc6.AvL <- lfc6.AvL %>% as.data.frame() %>% rownames_to_column("orfs") 
write.csv(lfc6.AvL, "./de_res_files/lfc6.AvL.csv", row.names=F)

lfc13.AvL <- lfcShrink(de13,  contrast=c("treatment", "Add_Back", "Low_Iron"), type='ashr')
lfc13.AvL <- lfc13.AvL %>% as.data.frame() %>% rownames_to_column("orfs") 
write.csv(lfc13.AvL, "./de_res_files/lfc13.AvL.csv", row.names=F)

lfc8.HvL <- lfcShrink(de8, contrast=c("treatment", "High_Iron", "Low_Iron"), type='ashr')
lfc8.HvL <- lfc8.HvL %>% as.data.frame() %>% rownames_to_column("orfs") 
write.csv(lfc8.HvL, "./de_res_files/lfc8.HvL.csv", row.names=F)

lfc6.HvL <- lfcShrink(de6, contrast=c("treatment", "High_Iron", "Low_Iron"), type='ashr')
lfc6.HvL <- lfc6.HvL %>% as.data.frame() %>% rownames_to_column("orfs") 
write.csv(lfc6.HvL, "./de_res_files/lfc6.HvL.csv", row.names=F)

lfc13.HvL <- lfcShrink(de13,  contrast=c("treatment", "High_Iron", "Low_Iron"), type='ashr')
lfc13.HvL <- lfc13.HvL %>% as.data.frame() %>% rownames_to_column("orfs") 
write.csv(lfc13.HvL, "./de_res_files/lfc13.HvL.csv", row.names=F)

lfc8.AvH <- lfcShrink(de8, contrast=c("treatment", "Add_Back", "High_Iron"), type='ashr')
lfc8.AvH <- lfc8.AvH %>% as.data.frame() %>% rownames_to_column("orfs") 
write.csv(lfc8.AvH, "./de_res_files/lfc8.AvH.csv", row.names=F)

lfc6.AvH <- lfcShrink(de6, contrast=c("treatment", "Add_Back", "High_Iron"), type='ashr')
lfc6.AvH <- lfc6.AvH %>% as.data.frame() %>% rownames_to_column("orfs") 
write.csv(lfc6.AvH, "./de_res_files/lfc6.AvH.csv", row.names=F)

lfc13.AvH <- lfcShrink(de13,  contrast=c("treatment", "Add_Back", "High_Iron"), type='ashr')
lfc13.AvH <- lfc13.AvH %>% as.data.frame() %>% rownames_to_column("orfs") 
write.csv(lfc13.AvH, "./de_res_files/lfc13.AvH.csv", row.names=F)

using 'ashr' for LFC shrinkage. If used in published research, please cite:
    Stephens, M. (2016) False discovery rates: a new deal. Biostatistics, 18:2.
    https://doi.org/10.1093/biostatistics/kxw041

using 'ashr' for LFC shrinkage. If used in published research, please cite:
    Stephens, M. (2016) False discovery rates: a new deal. Biostatistics, 18:2.
    https://doi.org/10.1093/biostatistics/kxw041

using 'ashr' for LFC shrinkage. If used in published research, please cite:
    Stephens, M. (2016) False discovery rates: a new deal. Biostatistics, 18:2.
    https://doi.org/10.1093/biostatistics/kxw041

using 'ashr' for LFC shrinkage. If used in published research, please cite:
    Stephens, M. (2016) False discovery rates: a new deal. Biostatistics, 18:2.
    https://doi.org/10.1093/biostatistics/kxw041

using 'ashr' for LFC shrinkage. If used in published research, please cite:
    Stephens, M. (2016) False discovery rates: a new deal. Biostatistics, 18:2.
    https://doi.org/

## 2.4 Subset results tables by significance and save
df's made into a list and looped through to order rows by adjusted p value and pull out significant rows. saving both files. Sorting the _res files positive or negative log2fold change will enable me to get the up regulated and down regulated genes.

In [13]:
df.names <- list()
samples <- c('4','8','6','13')
contrasts <- c('AvL','HvL','AvH')

for (s in samples){
    for (c in contrasts){
        if (exists(paste(c,s, sep="")) == FALSE){
            print(paste('contrast ', c, ' for ', s, ' not found'))
            next}
        df.names <- append(df.names, paste(s,c,sep=''))
    }
}

res.ls <- list(AvL4, AvL8, AvL6, AvL13, HvL8, HvL6, HvL13, AvH8, AvH6, AvH13)
names(res.ls) <- df.names

res.ls <- lapply(res.ls, function(df){   #order each df in list by p.value
    arrange(df, padj)
})

#save each df ordered by p.value
walk2(res.ls, paste0("./de_res_files/", names(res.ls), "_res.csv", sep=""), write.csv,row.names=F)

res.ls.sig <- lapply(res.ls, function(df){   #pull out significant DE's
    filter(df, padj<=0.05)
})

#save significant de's for each df
walk2(res.ls.sig, paste0("./de_res_files/", names(res.ls), "sig_res.csv", sep=""), write.csv,row.names=F)

[1] "contrast  HvL  for  4  not found"
[1] "contrast  AvH  for  4  not found"


# Glimma MA plots

The Glimma MDS contains two main components:

1. a plot showing two MDS dimensions, and
2. a plot of the eigenvalues of each dimension

The Glimma MA plot contains two main components:

1. a plot of summary statistics across all genes that have been tested, and
2. a plot of gene expression from individual samples for a given gene

In [ ]:
glimmaMDS(dds4.ko)

In [ ]:
de4.ko <- DESeq(dds4.ko)
de8.ko <- DESeq(dds8.ko)
de6.ko <- DESeq(dds6.ko)
de13.ko <- DESeq(dds13.ko)

In [ ]:

ma.glimma <- function(de){
    #plotting function removes orfs with a pvalue of na, its not updating the anno df properly, 
    #so here I calculated those which will be removed
    #and make a df with the annotations already removed
    r <- as.data.frame(results(de))
    complete.genes <- (complete.cases(r))
    res.comp <- r[complete.genes, ]
    x <- de[complete.genes, ]
   
    glimmaMA(de,
            #status=NULL,                                    # vector indicated status of each gene
             groups= colData(de)$treatment,    #vector length of ncol dds, shows categories of samples 
             anno=mcols(x),                                         #df with nrow of de object with rows containing gene annotations
             display.columns=c('symbol','name'),  #character vector with names from anno from which to display 
             main='MA Plot', 
             xlab='logCPM', 
             ylab='log2FC')
    }

In [ ]:
ma.glimma(de4)

In [ ]:
ma.glimma(de8)

In [ ]:
ma.glimma(de6)

In [ ]:
ma.glimma(de13)

# Make table of % kegg annotated orfs and other functional annotation info

In [20]:
ko4_ls=read.csv('../kegg_names/ko4_ls.csv')
ko8_ls=read.csv('../kegg_names/ko8_ls.csv')
ko6_ls=read.csv('../kegg_names/ko6_ls.csv')
ko13_ls=read.csv('../kegg_names/ko13_ls.csv')


#make sure to REMOVE ORFS NOT COUNTED BY SALMON!
ko_annotated=data.frame('Organism'=c('C. closterium 4', 'C. closterium 8', 'G. oceanica', 'G. huxleyi'),
                        'Anotated orfs (%)'=c(
                            (nrow(filter(ko4_ls, ko_iteration=='ko:_1'))/nrow(de4)*100),
                            (nrow(filter(ko8_ls, ko_iteration=='ko:_1'))/nrow(de8)*100),
                            (nrow(filter(ko6_ls, ko_iteration=='ko:_1'))/nrow(de6)*100),
                            (nrow(filter(ko13_ls, ko_iteration=='ko:_1'))/nrow(de13)*100)))
head(ko_annotated)
write.csv(ko_annotated, './de_res_files/ko_anotat_de.csv', row.names=F)

[1] "% orfs annotated"


,Organism,Anotated.orfs....
,<chr>,<dbl>
1,C. closterium 4,45.27044
2,C. closterium 8,46.33550
3,G. oceanica,66.00849
4,G. huxleyi,70.26608


# Now repeat steps but summarize to 'kegg gene level'
## 1. Make df of raw counts which are summed to kegg level.
#### Read in the list of ORF-to-Ko's for each sample 
This list repeates orfs when multiple ko's were assigned to a single orf by eggnog. Using the 'orf' column from the organism specific ko list, we want to map the counts to each row of the orf-to-ko list. This will automatically repeat the counts for each repeated orf in the list and allow us to sum orfs with matching ko's later. 

<b/> Remember, not all rows from the counts table will have a ko assigned and thus will not appear in the orf-to-ko list. Additionally, not all orfs annotated by eggnog were counted by salmon. Thus we must remove any rows from the orf-to-ko table for which orfs are not found </b>

Because some orfs had multiple ko assignments the merging relationship will be many-to-one, many orf-ko rows matching to one counts row. "Each row in x (orf-to-ko list) matches at most 1 row in y (counts table)."

Once the two tables are merged, we can group by ko_id and sum counts which have the same ko_id. The result is a new raw counts table (matrix) which we can used in deseq. 

In [6]:
ko.def=read.csv('../kegg_names/ko_def.csv')
sum.kegg <- function(org, counts.raw){
    ko_df= read.csv(paste('../kegg_names/ko', org,'_ls.csv', sep=''))
    #ko_df = organsim-specific orf-to-ko list
    ko_df <- select(ko_df, c('orfs', 'ko_id'))
    
    # make raw counts matrix into a tibble
    counts <- as_tibble(counts.raw$counts, rownames = "orfs")
    
    # remove orfs that eggnog annotated but salmon did not count
    ko <- filter(ko_df, (ko_df$orfs %in% counts$orfs)==TRUE)
    
    # remove orfs that did not have a matching kegg annotation
    #counts.ko <- filter(counts, (counts$orfs%in%ko_df$orfs)==TRUE)

    
    # merge the two, so orfs and thier counts are repeated when they match to multiple ko's
    b <- left_join(x=ko, y=counts, by='orfs', relationship="many-to-one")
    if(all(ko$orfs%in% counts$orfs)==FALSE){
        print(paste("ERROR: merged counts and ko_ids should be the same length as 
                    the ko_id df length of merged counts is ", nrow(b), " and ko_ids is: ", 
                    nrow(ko), sep=''))
    }

    #group by ko_id and sum counts for each ko_id
    b <- b %>% select(!orfs) %>% 
        group_by(ko_id) %>% 
        summarize(across(everything(), sum)) %>%
        column_to_rownames("ko_id") %>% 
        as.matrix
    
    mode(b) <- 'integer'
    print(' # unique ko_ids should equal number rows of ko-summed counts.')
    print('# unique ko_ids = ')
    print(length(unique(ko$ko_id)))
    print(' and # ko-summed counts = ')
    print(nrow(b))
    b
    }

kcounts_4 <- sum.kegg('4', counts_4)
kcounts_8 <- sum.kegg('8', counts_8)
kcounts_6 <- sum.kegg('6', counts_6)
kcounts_13 <- sum.kegg('13', counts_13)


[1] " # unique ko_ids should equal number rows of ko-summed counts."
[1] "# unique ko_ids = "
[1] 4243
[1] " and # ko-summed counts = "
[1] 4243
[1] " # unique ko_ids should equal number rows of ko-summed counts."
[1] "# unique ko_ids = "
[1] 4436
[1] " and # ko-summed counts = "
[1] 4436
[1] " # unique ko_ids should equal number rows of ko-summed counts."
[1] "# unique ko_ids = "
[1] 5374
[1] " and # ko-summed counts = "
[1] 5374
[1] " # unique ko_ids should equal number rows of ko-summed counts."
[1] "# unique ko_ids = "
[1] 5270
[1] " and # ko-summed counts = "
[1] 5270


## 2. Run DeSeq2 on kegg-summed counts
Using DESeqDataSetFromMatrix, read in summed counts matrix.

Next, add the ko name and symbol to the metadata for each deseq object. Run the DE and VST normalization 

In [7]:
## create deseq object from matrix
k.dds <- function(k.counts, metadata,n){
    #n = lowest number of reps in any treatment
   dds <-  DESeqDataSetFromMatrix(
       countData = k.counts, 
       colData = metadata, 
       design=~treatment)
    dds$treatment <- relevel(dds$treatment, ref = "Low_Iron")
    keep <- rowSums(counts(dds) >=10) >= n #filter out rows with too low expression
    dds <- dds[keep, ]
    dds}

k.dds4 <- k.dds(kcounts_4, metadata_4, 2)
k.dds8 <- k.dds(kcounts_8, metadata_8, 2)
k.dds6 <- k.dds(kcounts_6, metadata_6, 3)
k.dds13 <- k.dds(kcounts_13, metadata_13, 3)

add.rowData <- function(dds, ko_def){
    d <- data.frame('ko_id'= rownames(dds))
        #filter ko's which are present in the dds df
    v <- ko_def[(ko_def$ko_id%in%d$ko_id)==TRUE, c('ko_id', 'symbol', 'name')]
        #because counts are summed by ko_id, I need to take only the unique ko's from anno
    anno <- distinct(v)
    all(rownames(dds)==anno$ko_id)
    #reorder annotation table to same row order as dds
    anno <- anno[match(rownames(dds), anno$ko_id),]
    all(rownames(dds)==anno$ko_id)

    mcols(dds) <- cbind(mcols(dds), anno)
    print(mcols(dds))
}

#mcols(k.dds4) <- add.rowData(k.dds4, ko4_def)
#mcols(k.dds8) <- add.rowData(k.dds8, ko8_def)
#mcols(k.dds6) <- add.rowData(k.dds6, ko6_def)
#mcols(k.dds13) <- add.rowData(k.dds13, ko13_def)
#mcols(k.dds13)
    
## vsd normalize and save ordered by variance
vsd.norm <- function(dds){
    varianceStabilizingTransformation(dds,blind=FALSE)
    }
k.vsd4 <- vsd.norm(k.dds4)
k.vsd8 <- vsd.norm(k.dds8)
k.vsd6 <- vsd.norm(k.dds6)
k.vsd13 <- vsd.norm(k.dds13)
#k.vsd4 <- varianceStabilizingTransformation(k.dds4,blind = FALSE)
write.vsd.k <- function(vsd, org){
    vsd <- assay(vsd)
    vsd_order <- order(rowVars(vsd), decreasing=T)
    vsd_new <- vsd[vsd_order, ]
    write.csv(vsd_new, paste('./vsd_files/', org, "vsd.k.csv", sep=""))
}

write.vsd.k(k.vsd4, '04')
write.vsd.k(k.vsd8, '08')
write.vsd.k(k.vsd6, '06')
write.vsd.k(k.vsd13, '13')

## run differential expression analysis
de4.k <- DESeq(k.dds4)
de8.k <- DESeq(k.dds8)
de6.k <- DESeq(k.dds6)
de13.k <- DESeq(k.dds13)

#extract results
HvL <- c("treatment", "High_Iron", "Low_Iron")
AvL <- c("treatment", "Add_Back", "Low_Iron")
AvH <- c("treatment", "Add_Back", "High_Iron")

#results from Low Iron vs Iron ammendment
AvL4.k <- results(de4.k, contrast=AvL, tidy=TRUE)
AvL8.k <- results(de8.k, contrast=AvL, tidy=T)
AvL6.k <- results(de6.k, contrast=AvL, tidy=TRUE)
AvL13.k <- results(de13.k, contrast=AvL, tidy=TRUE)

#results from Low Iron vs Iron ammendment
AvL4.k <- results(de4.k, contrast=AvL, tidy=TRUE)
AvL8.k <- results(de8.k, contrast=AvL, tidy=T)
AvL6.k <- results(de6.k, contrast=AvL, tidy=TRUE)
AvL13.k <- results(de13.k, contrast=AvL, tidy=TRUE)

#results from High Iron vs Low Iron
HvL8.k  <- results(de8.k, contrast=HvL, tidy=TRUE)
HvL6.k  <- results(de6.k, contrast=HvL,  tidy=TRUE)
HvL13.k  <- results(de13.k,contrast=HvL, tidy=TRUE)

#results from High Iron vs Iron ammendment
AvH8.k  <- results(de8.k, contrast=AvH, tidy=TRUE)
AvH6.k  <- results(de6.k, contrast=AvH,  tidy=TRUE)
AvH13.k  <- results(de13.k, contrast=AvH, tidy=TRUE)

#save deseq results

AvL4.k %>% na.omit() %>% write.csv('./de_res_files/AvL4.k.csv', row.names=F)
AvL8.k%>% na.omit() %>% write.csv('./de_res_files/AvL8.k.csv', row.names=F)
AvL6.k %>% na.omit() %>% write.csv('./de_res_files/AvL6.k.csv', row.names=F)
AvL13.k %>% na.omit() %>% write.csv('./de_res_files/AvL13.k.csv', row.names=F)

AvH8.k %>% na.omit() %>% write.csv('./de_res_files/AvH8.k.csv', row.names=F)
AvH6.k %>% na.omit() %>% write.csv('./de_res_files/AvH6.k.csv', row.names=F)
AvH13.k %>% na.omit() %>% write.csv('./de_res_files/AvH13.k.csv', row.names=F)

HvL8.k %>% na.omit() %>% write.csv('./de_res_files/HvL8.k.csv', row.names=F)
HvL6.k %>% na.omit() %>% write.csv('./de_res_files/HvL6.k.csv', row.names=F)
HvL13.k %>% na.omit() %>% write.csv('./de_res_files/HvL13.k.csv', row.names=F)


Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

f

In [12]:
colData(de8.k)

DataFrame with 8 rows and 5 columns
                   id     isolate treatment         rep sizeFactor
          <character> <character>  <factor> <character>  <numeric>
add_backB   add_backB          08 Add_Back            B   2.045848
add_backC   add_backC          08 Add_Back            C   1.056302
pFe21_9A     pFe21_9A          08 Low_Iron            A   0.599845
pFe19A         pFe19A          08 High_Iron           A   0.713617
pFe19B         pFe19B          08 High_Iron           B   0.592784
pFe19C         pFe19C          08 High_Iron           C   1.221984
pFe21_9B     pFe21_9B          08 Low_Iron            B   1.350000
pFe21_9C     pFe21_9C          08 Low_Iron            C   1.705215

In [25]:
# how many DE's per treatment-sample?

avl4.perc.k=(nrow(filter(AvL4.k,(padj < 0.05)==T))/nrow(de4.k))*100
avl4.k=nrow(filter(AvL4.k,(padj < 0.05)==T))

avl8.perc.k=(nrow(filter(AvL8.k,(padj < 0.05)==T))/nrow(de8.k))*100
avl8.k=nrow(filter(AvL8.k,(padj < 0.05)==T))

avl6.perc.k=(nrow(filter(AvL6.k,(padj < 0.05)==T))/nrow(de6.k))*100
avl6.k=nrow(filter(AvL6.k,(padj < 0.05)==T))

avl13.perc.k=(nrow(filter(AvL13.k,(padj < 0.05)==T))/nrow(de13.k))*100
avl13.k=nrow(filter(AvL13.k,(padj < 0.05)==T))


hvl8.perc.k=(nrow(filter(HvL8.k,(padj < 0.05)==T))/nrow(de8.k))*100
hvl8.k=nrow(filter(HvL8.k,(padj < 0.05)==T))

hvl6.perc.k=(nrow(filter(HvL6.k,(padj < 0.05)==T))/nrow(de6.k))*100
hvl6.k=nrow(filter(HvL6.k,(padj < 0.05)==T))

hvl13.perc.k=(nrow(filter(HvL13.k,(padj < 0.05)==T))/nrow(de13.k))*100
hvl13.k=nrow(filter(HvL13.k,(padj < 0.05)==T))


avh8.perc.k=(nrow(filter(AvH8.k,(padj < 0.05)==T))/nrow(de8.k))*100
avh8.k=nrow(filter(AvH8.k,(padj < 0.05)==T))

avh6.perc.k=(nrow(filter(AvH6.k,(padj < 0.05)==T))/nrow(de6.k))*100
avh6.k=nrow(filter(AvH6.k,(padj < 0.05)==T))

avh13.perc.k=(nrow(filter(AvH13.k,(padj < 0.05)==T))/nrow(de13.k))*100
avh13.k=nrow(filter(AvH13.k,(padj < 0.05)==T))

de_ko_output = data.frame('Organism'=c('C. closterium 4', 'C. closterium 8', 'G. oceanica', 'G. huxleyi'),
                       'Iron amendment vs low iron'=c(avl4.k,avl8.k,avl6.k,avl13.k),
                       'High iron vs low iron'=c('NA', hvl8.k,hvl6.k,hvl13.k),
                       'Iron amendment vs high iron'=c('NA',avh8.k,avh6.k,avh13.k))

de_ko_output_perc = data.frame('Organism'=c('C. closterium 4', 'C. closterium 8', 'G. oceanica', 'G. huxleyi'),
                       'Iron amendment vs low iron'=c(avl4.perc.k,avl8.perc.k,avl6.perc.k,avl13.perc.k),
                       'High iron vs low iron'=c('NA', hvl8.perc.k,hvl6.perc.k,hvl13.perc.k),
                       'Iron amendment vs high iron'=c('NA',avh8.perc.k,avh6.perc.k,avh13.perc.k)) 

write.csv(de_ko_output, './de_res_files/de_ko_output.csv', row.names=F)
write.csv(de_ko_output_perc, './de_res_files/de_ko_output_perc.csv', row.names=F)

head(de_ko_output)

,Organism,Iron.amendment.vs.low.iron,High.iron.vs.low.iron,Iron.amendment.vs.high.iron
,<chr>,<int>,<chr>,<chr>
1,C. closterium 4,206,NA,NA
2,C. closterium 8,830,300,947
3,G. oceanica,1291,1381,221
4,G. huxleyi,1154,1132,396


In [46]:
#Save info for top 30 VSD normalized orfs with highest variance across samples

top30vsd.info <- function(k.vsd, ko_def, org){
    #order vsd matrix by variance across samples
    vsd <- assay(k.vsd)
    var_order <- order(rowVars(vsd), decreasing=T)
    vsd_new <- vsd[var_order, ]
    vsd_new <- as.data.frame(vsd_new) %>% rownames_to_column("ko_id") 
    #select top 30 orfs, these have the highest variance
    vsd30 <- vsd_new[1:30,]
    #merge with name and info for ko id and create seperate column of enzyme info
    vsd30 <- left_join(vsd30, ko.def, by = 'ko_id') %>% select('ko_id', 'symbol', 'name')
    vsd30 <- vsd30 %>% separate(name, c('name', 'enzyme'), "\\[EC:")  
    vsd30$enzyme <- str_replace(vsd30$enzyme, "\\]", "")
    vsd30
    write.csv(vsd30, paste("./vsd_files/vsd.30.", org, ".csv", sep=''), row.names=F)
}

top30vsd.info(k.vsd4, ko4_def, "4")
top30vsd.info(k.vsd8, ko8_def, "8")
top30vsd.info(k.vsd6, ko6_def, "6")
top30vsd.info(k.vsd13, ko13_def, "13")


Warning message:
“Expected 2 pieces. Missing pieces filled with `NA` in 17 rows [1, 2, 3, 5, 7, 8, 9, 10, 13, 17, 22, 24,
26, 27, 28, 29, 30].”
Warning message:
“Expected 2 pieces. Missing pieces filled with `NA` in 15 rows [2, 3, 4, 8, 9, 11, 12, 14, 15, 16, 20, 24,
26, 29, 30].”
Warning message:
“Expected 2 pieces. Missing pieces filled with `NA` in 14 rows [1, 4, 5, 6, 7, 8, 11, 12, 20, 21, 22, 27,
29, 30].”
Warning message:
“Expected 2 pieces. Missing pieces filled with `NA` in 18 rows [1, 3, 5, 6, 7, 10, 11, 12, 15, 17, 18, 20,
21, 22, 23, 24, 26, 29].”


In [8]:
lfc4.k.AvL <- lfcShrink(de4.k,  contrast=c("treatment", "Add_Back", "Low_Iron"), type='ashr')
lfc4.k.AvL <- lfc4.k.AvL %>% as.data.frame() %>% rownames_to_column("ko_id") 
lfc8.k.AvL <- lfcShrink(de8.k, contrast=c("treatment", "Add_Back", "Low_Iron"), type='ashr')
lfc8.k.AvL <- lfc8.k.AvL %>% as.data.frame() %>% rownames_to_column("ko_id") 
lfc6.k.AvL <- lfcShrink(de6.k, contrast=c("treatment", "Add_Back", "Low_Iron"), type='ashr')
lfc6.k.AvL <- lfc6.k.AvL %>% as.data.frame() %>% rownames_to_column("ko_id") 
lfc13.k.AvL <- lfcShrink(de13.k,  contrast=c("treatment", "Add_Back", "Low_Iron"), type='ashr')
lfc13.k.AvL <- lfc13.k.AvL %>% as.data.frame() %>% rownames_to_column("ko_id") 


lfc8.k.HvL <- lfcShrink(de8.k, contrast=c("treatment", "High_Iron", "Low_Iron"), type='ashr')
lfc8.k.HvL <- lfc8.k.HvL %>% as.data.frame() %>% rownames_to_column("ko_id") 
lfc6.k.HvL <- lfcShrink(de6.k, contrast=c("treatment", "High_Iron", "Low_Iron"), type='ashr')
lfc6.k.HvL <- lfc6.k.HvL %>% as.data.frame() %>% rownames_to_column("ko_id") 
lfc13.k.HvL <- lfcShrink(de13.k,  contrast=c("treatment", "High_Iron", "Low_Iron"), type='ashr')
lfc13.k.HvL <- lfc13.k.HvL %>% as.data.frame() %>% rownames_to_column("ko_id") 

lfc8.k.AvH <- lfcShrink(de8.k, contrast=c("treatment", "Add_Back", "High_Iron"), type='ashr')
lfc8.k.AvH <- lfc8.k.AvH %>% as.data.frame() %>% rownames_to_column("ko_id") 
lfc6.k.AvH <- lfcShrink(de6.k, contrast=c("treatment", "Add_Back", "High_Iron"), type='ashr')
lfc6.k.AvH <- lfc6.k.AvH %>% as.data.frame() %>% rownames_to_column("ko_id")
lfc13.k.AvH <- lfcShrink(de13.k,  contrast=c("treatment", "Add_Back", "High_Iron"), type='ashr')
lfc13.k.AvH <- lfc13.k.AvH %>% as.data.frame() %>% rownames_to_column("ko_id") 

write.csv(lfc4.k.AvL, "./de_res_files/lfc4.k.AvL.csv", row.names=F)
write.csv(lfc8.k.AvL, "./de_res_files/lfc8.k.AvL.csv", row.names=F)
write.csv(lfc6.k.AvL, "./de_res_files/lfc6.k.AvL.csv", row.names=F)
write.csv(lfc13.k.AvL, "./de_res_files/lfc13.k.AvL.csv", row.names=F)
write.csv(lfc8.k.HvL, "./de_res_files/lfc8.k.HvL.csv", row.names=F)
write.csv(lfc6.k.HvL, "./de_res_files/lfc6.k.HvL.csv", row.names=F)
write.csv(lfc13.k.HvL, "./de_res_files/lfc13.k.HvL.csv", row.names=F)
write.csv(lfc8.k.AvH, "./de_res_files/lfc8.k.AvH.csv", row.names=F)
write.csv(lfc6.k.AvH, "./de_res_files/lfc6.k.AvH.csv", row.names=F)
write.csv(lfc13.k.AvH, "./de_res_files/lfc13.k.AvH.csv", row.names=F)

using 'ashr' for LFC shrinkage. If used in published research, please cite:
    Stephens, M. (2016) False discovery rates: a new deal. Biostatistics, 18:2.
    https://doi.org/10.1093/biostatistics/kxw041

using 'ashr' for LFC shrinkage. If used in published research, please cite:
    Stephens, M. (2016) False discovery rates: a new deal. Biostatistics, 18:2.
    https://doi.org/10.1093/biostatistics/kxw041

using 'ashr' for LFC shrinkage. If used in published research, please cite:
    Stephens, M. (2016) False discovery rates: a new deal. Biostatistics, 18:2.
    https://doi.org/10.1093/biostatistics/kxw041

using 'ashr' for LFC shrinkage. If used in published research, please cite:
    Stephens, M. (2016) False discovery rates: a new deal. Biostatistics, 18:2.
    https://doi.org/10.1093/biostatistics/kxw041

using 'ashr' for LFC shrinkage. If used in published research, please cite:
    Stephens, M. (2016) False discovery rates: a new deal. Biostatistics, 18:2.
    https://doi.org/

In [22]:
ranked4.AvL = lfc4.k.AvL[order(abs(lfc4.k.AvL$log2FoldChange), decreasing=TRUE),]
ranked4.AvL = left_join(ranked4.AvL, ko.def)


ranked8.AvL = lfc8.k.AvL[order(abs(lfc8.k.AvL$log2FoldChange), decreasing=TRUE),]
ranked8.AvL = left_join(ranked8.AvL, ko.def)

ranked8.HvL = lfc8.k.HvL[order(abs(lfc8.k.HvL$log2FoldChange), decreasing=TRUE),]
ranked8.HvL = left_join(ranked8.HvL, ko.def)

ranked8.AvH = lfc8.k.AvH[order(abs(lfc8.k.AvH$log2FoldChange), decreasing=TRUE),]
ranked8.AvH = left_join(ranked8.AvH, ko.def)


ranked6.AvL = lfc6.k.AvL[order(abs(lfc6.k.AvL$log2FoldChange), decreasing=TRUE),]
ranked6.AvL = left_join(ranked6.AvL, ko.def)

ranked6.HvL = lfc6.k.HvL[order(abs(lfc6.k.HvL$log2FoldChange), decreasing=TRUE),]
ranked6.HvL = left_join(ranked6.HvL, ko.def)

ranked6.AvH = lfc6.k.AvH[order(abs(lfc6.k.AvH$log2FoldChange), decreasing=TRUE),]
ranked6.AvH = left_join(ranked6.AvH, ko.def)


ranked13.AvL = lfc13.k.AvL[order(abs(lfc13.k.AvL$log2FoldChange), decreasing=TRUE),]
ranked13.AvL = left_join(ranked13.AvL, ko.def)

ranked13.HvL = lfc13.k.HvL[order(abs(lfc13.k.HvL$log2FoldChange), decreasing=TRUE),]
ranked13.HvL = left_join(ranked13.HvL, ko.def)

ranked13.AvH = lfc13.k.AvH[order(abs(lfc13.k.AvH$log2FoldChange), decreasing=TRUE),]
ranked13.AvH = left_join(ranked13.AvH, ko.def)

head(ranked13.AvH)
write.csv(ranked4.AvL, "./de_res_files/ranked4.AvL.csv", row.names=F)
write.csv(ranked8.AvL, "./de_res_files/ranked8.AvL.csv", row.names=F)
write.csv(ranked6.AvL, "./de_res_files/ranked6.AvL.csv", row.names=F)
write.csv(ranked13.AvL, "./de_res_files/ranked13.AvL.csv", row.names=F)
write.csv(ranked8.AvH, "./de_res_files/ranked8.AvH.csv", row.names=F)
write.csv(ranked6.AvH, "./de_res_files/ranked6.AvH.csv", row.names=F)
write.csv(ranked13.AvH, "./de_res_files/ranked13.AvH.csv", row.names=F)
write.csv(ranked8.HvL, "./de_res_files/ranked8.HvL.csv", row.names=F)
write.csv(ranked6.HvL, "./de_res_files/ranked6.HvL.csv", row.names=F)
write.csv(ranked13.HvL, "./de_res_files/ranked13.HvL.csv", row.names=F)

Joining with `by = join_by(ko_id)`
Joining with `by = join_by(ko_id)`
Joining with `by = join_by(ko_id)`
Joining with `by = join_by(ko_id)`
Joining with `by = join_by(ko_id)`
Joining with `by = join_by(ko_id)`
Joining with `by = join_by(ko_id)`
Joining with `by = join_by(ko_id)`
Joining with `by = join_by(ko_id)`
Joining with `by = join_by(ko_id)`


,ko_id,baseMean,log2FoldChange,lfcSE,pvalue,padj,symbol,name
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
1,K00525,1073.92643,-3.904022,0.2805200,8.315263e-46,1.930804e-42,"E1.17.4.1A, nrdA, nrdE",ribonucleoside-diphosphate reductase alpha chain [EC:1.17.4.1]
2,K10734,25.89301,-3.212495,0.5927836,1.303292e-09,1.061840e-07,GINS3,GINS complex subunit 3
3,K10808,501.63986,-3.196746,0.2288487,1.401861e-45,2.170081e-42,RRM2,ribonucleoside-diphosphate reductase subunit M2 [EC:1.17.4.1]
4,K18710,22.68003,-3.160268,0.8150535,4.954375e-07,2.130381e-05,SLBP,histone RNA hairpin-binding protein
5,K22156,56.82509,-3.085140,0.4034052,9.545449e-16,1.528588e-13,JADE3,protein Jade-3
6,K14291,83.10115,-3.078023,0.3364991,3.207594e-21,8.762392e-19,PHAX,phosphorylated adapter RNA export protein


In [36]:
module.url <- "https://rest.kegg.jp/link/module/ko04050"
    module.url <- getURL(module.url)
    module.link <- read.csv((module.url), sep = '\t', header = F)  #find brite categories associated with ko
    

Warning message in file(file, "rt"):
“cannot open file '
': No such file or directory”


ERROR: Error in file(file, "rt"): cannot open the connection


In [34]:
ko.30 <- list(vsd4.30$ko_id)

module.list <- list()
for (i in ko.30){
    module.url <-  paste0("https://rest.kegg.jp/link/module/",i,sep='')
    module.url <- getURL(module.url)
    module.link <- read.csv(textConnection(module.url), sep = '\t', header = F)  #find brite categories associated with ko
    module.link$V2 <-  str_replace(module.link$V2, pattern = "md:" , replacement = '')
    module.link <- list(module.link$V2)
    append(module.list, module.link)
}

module.list
#for (i in module.list){
    name.url <- paste0("https://rest.kegg.jp/find/module/", 'ko00195', sep='')
    name.table <- getURL(name.url)
   name.table <-  read.csv(textConnection(name.table), sep='\t', header = F)
    print(name.table)
    #append(final, name.table)
#}

pathways <- list()
for (i in ko.30){
    pathway.url <-  paste0("https://rest.kegg.jp/link/pathway/",i,sep='')
    pathway.url <- getURL(pathway.url)
    pathway.link <- read.csv(textConnection(pathway.url), sep = '\t', header = F)  #find brite categories associated with ko
    pathway.link$V2 <-  str_replace(pathway.link$V2, pattern = "path:" , replacement = '')
    pathway.list <- list(pathway.link$V2)
    print(pathway.list)
    append(pathways, pathway.list)
}



for (i in pathway.list){
    name.url <- paste0("https://rest.kegg.jp/find/pathway/", i, sep='')
    name.table <- getURL(name.url)
   name.table <-  read.csv(textConnection(name.table), sep='\t', header = F)
    print(name.table)
    #append(final, name.table)
}



list()

ERROR: Error in read.table(file = file, header = header, sep = sep, quote = quote, : no lines available in input


# Glimma MA plots

The Glimma MDS contains two main components:

1. a plot showing two MDS dimensions, and
2. a plot of the eigenvalues of each dimension

The Glimma MA plot contains two main components:

1. a plot of summary statistics across all genes that have been tested, and
2. a plot of gene expression from individual samples for a given gene

In [ ]:
glimmaMDS(k.dds6)

In [ ]:

ma.glimma <- function(de, lfc.shrink){
    #plotting function removes orfs with a pvalue of na, its not updating the anno df properly, 
    #so here I calculated those which will be removed
    #and make a df with the annotations already removed
    r <- as.data.frame(results(de))
    complete.genes <- (complete.cases(r))
    res.comp <- r[complete.genes, ]
    x <- de[complete.genes, ]
    anno <- mcols(x)

    glimmaMA(de,
            #status=NULL,                                    # vector indicated status of each gene
             groups= colData(de)$treatment,    #vector length of ncol dds, shows categories of samples 
             anno=anno,                                         #df with nrow of de object with rows containing gene annotations
             #display.columns=c('symbol','name'),  #character vector with names from anno from which to display 
                 #type of transformation used
                                                                            #default is logcpm
             #status.cols=NULL,                           #vector of 3, indicateing CSS strings for color associated with 
                                                                          #status of -1, 0, 1
            #sample.cols=NULL,                     #character vector of length ncol(counts) with CSS strings for 
                                                                        #colors for each sample
             main='MA Plot', 
             xlab='logCPM', 
             ylab='log2FC')
            #html=NULL,                               #character string for naming HTML file for exporting widget ending in .html)
    }
ma.glimma(de4.k)

In [ ]:
lfc13.k.HvL <- lfcShrink(de13.k,  contrast=c("treatment", "High_Iron", "Low_Iron"), type='ashr')
mcols(lfc13.k.HvL)
mcols(de13.k)

In [ ]:
d <- data.frame('ko_id'= rownames(k.dds6))

v <- ko6_def%>% filter((ko_id %in% dds$ko_id)==TRUE) %>% select(ko_id, symbol, name)
anno <- distinct(v)
all(rownames(k.dds6)==anno$ko_id)
#reorder annotation table to same row order as dds
anno <- anno[match(rownames(k.dds6), anno$ko_id),]
all(rownames(k.dds6)==anno$ko_id)

mcols(k.dds6) <- cbind(mcols(k.dds6), anno)
mcols(k.dds6)

## Now we have ordered vsd file and results file (for each comparison) for both transcript level and counts summarized at kegg annotation level. 
#### Taking these files we will create plots. 
## It might be best to use transcript level for ma plots and 'gene' level for clusterProfiler?